# Chain of Thought Prompting Example

This notebook demonstrates **Chain of Thought (CoT)**, one of the most fundamental and effective prompt engineering techniques.

**The Concept:**
Large Language Models (LLMs) often struggle with complex math, logic, or reasoning tasks if forced to give an immediate answer. **Chain of Thought** encourages the model to "show its work" by breaking the problem down into intermediate steps before arriving at the final solution.

**Key Mechanics:**
1.  **Instruction**: We explicitly tell the model to plan and execute steps.
2.  **Process**: The output will contain the reasoning path (The "Thought"), not just the result.
3.  **Result**: Accuracy on complex tasks increases significantly because the model "reasons" through the logic vector.

In [ ]:
%pip install openai python-dotenv --quiet

### 1. Setup and Authorization

We start by importing the `openai` library and securely loading your API key.

In [ ]:
from openai import OpenAI
import os
import time
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    api_key = input("Paste your OpenAI API key: ").strip()

# Model configuration - can be overridden via environment variable
MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")

client = OpenAI(api_key=api_key)
print(f"OpenAI client ready! Using model: {MODEL}")

### 2. The System Prompt (The "Instructions")

This is where we implement the technique. A standard prompt might just say "You are a helpful assistant."

For **Chain of Thought**, we modify the system prompt to enforce a structured thinking process:
1.  **Understand**: Grasp the core request.
2.  **Plan**: Outline the steps needed.
3.  **Execute**: Perform the calculation or logic for each step.
4.  **Conclude**: Provide the final answer only after the reasoning is complete.

This significantly reduces "hallucinations" and calculation errors.

In [ ]:
SYSTEM_PROMPT = """
You are a helpful assistant that solves problems using chain of thought reasoning.

For any query, do not just give the answer. Instead, break it down step by step:
1. Understand the problem: Restate what needs to be solved.
2. Plan the steps: Briefly list how you will approach it.
3. Execute: Go through each step with detailed explanations.
4. Final Answer: State the conclusion clearly at the end.
"""

### 3. The Interactive CoT Solver

In this loop, we send your complex problem to the model.

**Temperature Note**:
We use a low temperature (`0.2`) here.
* **Creative writing** benefits from high temperature (0.7–1.0).
* **Logical reasoning/Math** requires low temperature (0.0–0.3) to ensure the model chooses the most probable (logical) next token at every step of the chain.

In [ ]:
# Initialize the conversation with the CoT instructions
messages = [{"role": "system", "content": SYSTEM_PROMPT}]

# Maximum number of messages to keep (prevents token limit issues)
MAX_HISTORY = 20

def trim_message_history(messages, max_messages=MAX_HISTORY):
    """Keep system prompt and last N-1 messages to prevent token overflow."""
    if len(messages) > max_messages:
        return [messages[0]] + messages[-(max_messages - 1):]
    return messages

print("Chain of Thought Solver Ready! Enter a problem (e.g., 'If I have 3 apples...'). Type 'quit' to exit.\n")

while True:
    # 1. Get User Input
    user_input = input("Your problem: ")
    
    if user_input.strip().lower() in ["quit", "exit", "stop"]:
        print("Goodbye!")
        break
    
    if not user_input.strip():
        print("Please enter a valid problem.")
        continue
    
    # 2. Add to History
    messages.append({"role": "user", "content": user_input})
    
    # 3. Generate Reasoning
    try:
        # We use a low temperature to make the logic deterministic and focused.
        response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            temperature=0.2 
        )
        
        reply = response.choices[0].message.content
        
        # 4. Display the "Chain"
        print(f"\nReasoning Process:\n{reply}\n")
        print("-" * 60)
        
        # 5. Update History (Context)
        # This allows you to ask follow-up questions about the logic provided.
        messages.append({"role": "assistant", "content": reply})
        
        # 6. Trim history to prevent token overflow
        messages = trim_message_history(messages)
        
        # 7. Rate limiting to avoid API throttling
        time.sleep(0.2)
        
    except Exception as e:
        print(f"\nError: {e}\n")
        messages.pop()  # Remove the failed user message from history